# Exemplo de análise de agrupamentos

## Descrição dos Dados de Tráfego

| Campo | Tipo | Descrição |
|-------|------|-----------|
| ID | int32 | Identificador único do registro |
| vl_codigo | string | Código de identificação |
| UF | string | Unidade Federativa |
| REGIAO | string | Região geográfica |
| RELEVO_PRE | string | Tipo de relevo predominante |
| VELOCIDADE | double | Velocidade registrada? |
| CODIGO_BR | int32 | Código da rodovia federal |
| EXTENSAO | double | Extensão da via |
| SUPERFICIE | string | Tipo de superfície |
| JURISDICAO | string | Jurisdição da via |
| TIPO_LINK | string | Tipo de ligação/trecho |
| CLASSIFICACAO | string | Classificação da via |
| GEH | string | Indicador GEH |
| VMDA_C | double | Volume Médio Diário Anual - Crescente |
| VMDA_D | double | Volume Médio Diário Anual - Descrescente |
| VMDA_TOTAL | double | Volume Médio Diário Anual - Total |
| CODIGO_SNV.SRE | string | Código de referência SNV/SRE |
| Ano | int32 | Ano do registro |

In [1]:
################################################################################
# Definindo um caminho personalizado para bibliotecas R
################################################################################
# Exemplo de caminho: pode ser uma pasta dentro do seu diretório home
# Lembre-se que o R pode adicionar subdiretórios específicos da versão/arquitetura
meu_caminho_biblioteca <- "~/R/library"

# Define a variável de ambiente R_LIBS_USER para a sessão atual
Sys.setenv(R_LIBS_USER = meu_caminho_biblioteca)

# Verifica o valor da variável de ambiente
Sys.getenv("R_LIBS_USER")

# Mostra todos os caminhos de biblioteca que o R está usando
# O seu novo caminho deve aparecer aqui, geralmente como o primeiro da lista.
.libPaths()

[1] "~/R/library"

[1] "/usr/local/lib/R/site-library" "/usr/local/lib/R/library"     
[3] "/home/vscode/R/library"

In [2]:
################################################################################
#               INSTALAÇÃO E CARREGAMENTO DE PACOTES NECESSÁRIOS               #
################################################################################

# Pacotes utilizados
pacotes <- c("tidyverse", #carregar outros pacotes do R
             "knitr", "kableExtra", #formatação de tabelas
             "arrow","duckdb", # parquet e banco de dados
             "factoextra", "pheatmap") # visualização de agrupamentos

if(sum(as.numeric(!pacotes %in% installed.packages())) != 0){
  instalador <- pacotes[!pacotes %in% installed.packages()]
  for(i in 1:length(instalador)) {
    install.packages(instalador, dependencies = T, lib = meu_caminho_biblioteca)
    break()}
  sapply(pacotes, require, character = T, lib = meu_caminho_biblioteca) 
} else {
  sapply(pacotes, require, character = T, lib = meu_caminho_biblioteca) 
}

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: knitr

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘knitr’”
Loading required package: kableExtra


Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows


Loading required package: arrow


Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

 

tidyverse      knitr kableExtra      arrow     duckdb factoextra   pheatmap 
      TRUE      FALSE       TRUE       TRUE       TRUE       TRUE       TRUE

In [6]:
# Abrir dataset particionado de contagem de tráfego
df_trafego <- open_dataset(
  "Trabalho/bases-de-dados/contagem-de-trafego/contagem-de-trafego-pq/"
)
# Verificar esquema do dataset
cat("\n=== ESQUEMA DO DATASET DE TRÁFEGO ===\n")
print(df_trafego$schema)



=== ESQUEMA DO DATASET DE TRÁFEGO ===
Schema
ID: int32
vl_codigo: string
UF: string
REGIAO: string
RELEVO_PRE: string
VELOCIDADE: double
CODIGO_BR: int32
EXTENSAO: double
SUPERFICIE: string
JURISDICAO: string
TIPO_LINK: string
CLASSIFICACAO: string
GEH: string
VMDA_C: double
VMDA_D: double
VMDA_TOTAL: double
CODIGO_SNV.SRE: string
Ano: int32

See $metadata for additional Schema metadata


In [8]:
library(duckdb)
# Conectar DuckDB (in-memory, zero configuração)
con <- dbConnect(duckdb::duckdb())
# Abrir dataset Parquet particionado
ds <- open_dataset("Trabalho/bases-de-dados/contagem-de-trafego/contagem-de-trafego-pq/")
# Registrar como tabela virtual no DuckDB
duckdb_register(con, "dados", ds)
# SELECT COUNT(*), Ano GROUP BY Ano (SQL puro!)
resultado <- dbGetQuery(con, "
    SELECT *
    FROM dados 
    WHERE VMDA_TOTAL is not null
    LIMIT 15
")
print("cabeçalho do dataset de pavimentos:")
print(resultado)

[1] "cabeçalho do dataset de pavimentos:"
     ID  vl_codigo UF REGIAO RELEVO_PRE VELOCIDADE CODIGO_BR EXTENSAO
1  4322 307BAC0022 AC   <NA>       <NA>         NA       307      224
2  4323 307BAC0025 AC   <NA>       <NA>         NA       307      105
3  4324 307BAC0028 AC   <NA>       <NA>         NA       307        7
4  4325 307BAC0030 AC   <NA>       <NA>         NA       307       22
5  4536 317BAC0170 AC   <NA>       <NA>         NA       317      386
6  4537 317BAC0175 AC   <NA>       <NA>         NA       317       99
7  4538 317BAC0180 AC   <NA>       <NA>         NA       317       21
8  4539 317BAC0190 AC   <NA>       <NA>         NA       317      215
9  4540 317BAC0210 AC   <NA>       <NA>         NA       317       65
10 4541 317BAC0230 AC   <NA>       <NA>         NA       317      302
11 4542 317BAC0250 AC   <NA>       <NA>         NA       317      157
12 4543 317BAC0260 AC   <NA>       <NA>         NA       317       14
13 4544 317BAC0370 AC   <NA>       <NA>         

In [9]:
library(duckdb)
# Conectar DuckDB (in-memory, zero configuração)
con <- dbConnect(duckdb::duckdb())
# Abrir dataset Parquet particionado
ds <- open_dataset("Trabalho/bases-de-dados/condicoes-de-pavimento/condicoes-de-pavimento-pq/")
# Registrar como tabela virtual no DuckDB
duckdb_register(con, "dados", ds)
# SELECT COUNT(*), Ano GROUP BY Ano (SQL puro!)
resultado <- dbGetQuery(con, "
    SELECT *
    FROM dados 
    LIMIT 15
")
print("cabeçalho do dataset de pavimentos:")
print(resultado)

[1] "cabeçalho do dataset de pavimentos:"
        Contrato UF Rodovia km.inicial km.final Extensao.km       Data
1  01 00583/2018 AM  BR-230       2132      212         1.2 2020-11-30
2  01 00583/2018 AM  BR-230        212      211         1.0 2020-11-30
3  01 00583/2018 AM  BR-230        211      210         1.0 2020-11-30
4  01 00583/2018 AM  BR-230        210      209         1.0 2020-11-30
5  01 00583/2018 AM  BR-230        209      208         1.0 2020-11-30
6  01 00583/2018 AM  BR-230       2132      214         0.8 2020-11-30
7  01 00583/2018 AM  BR-230        214      215         1.0 2020-11-30
8  09 00346/2018 PR  BR-476       1977      198         0.3 2020-11-05
9  09 00346/2018 PR  BR-476        198      199         1.0 2020-11-05
10 09 00346/2018 PR  BR-476        199      200         1.0 2020-11-05
11 09 00346/2018 PR  BR-476        200      201         1.0 2020-11-05
12 09 00346/2018 PR  BR-476        201      202         1.0 2020-11-05
13 09 00346/2018 PR  BR-476        